In [1]:
import os
import pandas as pd
import geopandas as gpd
import numpy as np
import math
import re
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from os.path import join, getsize
%matplotlib inline
import plotly.graph_objects as go

ModuleNotFoundError: No module named 'plotly'

In [32]:
os.getcwd()
locdir = r'/Users/hmworsham/Desktop/RMBL/Projects/Watershed_Spatial_Dataset'
os.listdir(locdir)
snod2_inv = pd.read_csv(os.sep.join([locdir, r'Source/Snodgrass2_inventory_2020.csv']))
snod2_inv.head(5)

,Site_Name,Tag_Number,Sp,Height_1_M,Height_2_M,Height_Avg_M,DBH_2019,DBH_1_CM,DBH_2_CM,DBH_Avg_CM,...,Live_Dead,Health,Comments,Geo_Tagged,GeoTag_Association,Collection_Date,Latitude,Longitude,GPS_File_Name,Original_Tag_Year
0,snodgrass_2,5701,ABLA,12.4,12.6,12.50,22.1,NaN,NaN,NaN,...,Live,NaN,NaN,Yes,NaN,7/30/2020,NaN,NaN,WORSHAMM071610C,2019
1,snodgrass_2,5702,ABLA,9.4,9.5,9.45,15.5,NaN,NaN,NaN,...,Live,NaN,NaN,Yes,NaN,7/30/2020,NaN,NaN,WORSHAMM071808A,2019
2,snodgrass_2,5703,ABLA,6.8,6.5,6.65,10.6,NaN,NaN,NaN,...,Live,dead crown,NaN,Yes,NaN,7/30/2020,NaN,NaN,WORSHAMM071808A,2019
3,snodgrass_2,5704,PIEN,18.6,18.6,18.60,43.6,NaN,NaN,NaN,...,Live,NaN,NaN,Yes,NaN,7/30/2020,NaN,NaN,WORSHAMM071810B,2019
4,snodgrass_2,5705,ABLA,5.6,5.4,5.50,13.5,NaN,NaN,NaN,...,Live,NaN,NaN,Yes,NaN,7/30/2020,NaN,NaN,WORSHAMM071808A,2019


In [33]:
appd_dir = locdir + '/Scratch/RMBL_GPS_Data_MERGEDBYPLOT/'
appd_files = os.listdir(appd_dir)
appd_sf = [s for s in appd_files if s.endswith('.shp')]
appd_paths = [appd_dir + s for s in appd_sf]

appd_gpdf = []
for sf in appd_sf:
    gpdf = gpd.read_file(appd_dir + sf)
    appd_gpdf.append(gpdf)

In [34]:
snod2_gpdf = gpd.read_file(locdir + '/Scratch/Snod2_clean.shp')
snod2_gpdf.head(5)
print(snod2_gpdf.columns)

Index(['Study_Orga', 'Study_Loca', 'Project_Ty', 'Point_Type', 'Other',
       'Other2', 'Max_PDOP', 'Max_HDOP', 'Corr_Type', 'Rcvr_Type', 'GPS_Date',
       'GPS_Time', 'Update_Sta', 'Feat_Name', 'Datafile', 'Unfilt_Pos',
       'Filt_Pos', 'Data_Dicti', 'GPS_Week', 'GPS_Second', 'GNSS_Heigh',
       'Vert_Prec', 'Horz_Prec', 'Std_Dev', 'Latitude', 'Longitude',
       'Point_ID', 'Comment', 'Site', 'GPS_Sp_Cod', 'GPS_Tag_Nu', 'geometry'],
      dtype='object')


In [35]:
snod2_gpdf.columns
snod2_gpdf = snod2_gpdf.rename(columns = {'GPS_Tag_Nu':'Tag_Number'})

In [36]:
snod2_gpdf.head()

,Study_Orga,Study_Loca,Project_Ty,Point_Type,Other,Other2,Max_PDOP,Max_HDOP,Corr_Type,Rcvr_Type,...,Horz_Prec,Std_Dev,Latitude,Longitude,Point_ID,Comment,Site,GPS_Sp_Cod,Tag_Number,geometry
0,None,None,None,None,pien32,5781,207.4,99.9,Real-time SBAS Corrected,Geo 7X,...,1.3,1.037628,38.930236352,-106.986991006,1,None,Snodgrass-2,PIEN,5781,POINT (327767.685 4310911.225)
1,None,None,None,None,pien33,5782 - dead,141.3,99.9,Postprocessed Code,Geo 7X,...,1.4,0.143559,38.930247209,-106.986966692,2,None,Snodgrass-2,PIEN,5782,POINT (327769.819 4310912.384)
2,None,None,None,None,pien34,6036,2.7,1.4,Real-time SBAS Corrected,Geo 7X,...,1.3,1.086115,38.930261596,-106.986906179,6,None,Snodgrass-2,PIEN,6036,POINT (327775.100 4310913.866)
3,None,None,None,None,abla71,5789,5.4,2.4,Postprocessed Code,Geo 7X,...,1.4,0.698688,38.930279445,-106.986961123,8,None,Snodgrass-2,ABLA,5789,POINT (327770.380 4310915.951)
4,None,None,None,None,abla73,5788,2.6,1.8,Postprocessed Code,Geo 7X,...,0.6,0.188860,38.930297930,-106.986914548,10,None,Snodgrass-2,ABLA,5788,POINT (327774.462 4310917.914)


In [37]:
snod2_gpdf[snod2_gpdf['Tag_Number'] == 'abla']

,Study_Orga,Study_Loca,Project_Ty,Point_Type,Other,Other2,Max_PDOP,Max_HDOP,Corr_Type,Rcvr_Type,...,Horz_Prec,Std_Dev,Latitude,Longitude,Point_ID,Comment,Site,GPS_Sp_Cod,Tag_Number,geometry
212,None,None,None,None,5853 - same indiv as5854,abla199,5.4,2.7,Postprocessed Code,Geo 7X,...,0.9,0.355883,38.930322428,-106.986714227,27,None,Snodgrass-2,5853,abla,POINT (327791.887 4310920.255)


In [38]:
snod2_gpdf = snod2_gpdf.drop([13,212])

In [39]:
snod2_gpdf['Tag_Number'] = snod2_gpdf['Tag_Number'].astype('int64')

In [40]:
snod2_merge = snod2_inv.merge(snod2_gpdf, how = 'outer', on = 'Tag_Number')
snod2_merge_gpdf = gpd.GeoDataFrame(snod2_merge)

In [41]:
snod2_merge_gpdf.head(2)

,Site_Name,Tag_Number,Sp,Height_1_M,Height_2_M,Height_Avg_M,DBH_2019,DBH_1_CM,DBH_2_CM,DBH_Avg_CM,...,Vert_Prec,Horz_Prec,Std_Dev,Latitude_y,Longitude,Point_ID,Comment,Site,GPS_Sp_Cod,geometry
0,snodgrass_2,5701,ABLA,12.4,12.6,12.50,22.1,NaN,NaN,NaN,...,1.4,1.3,0.146559,38.930574210,-106.986959187,16.0,None,Snodgrass-2,ABLA,POINT (327771.261 4310948.661)
1,snodgrass_2,5702,ABLA,9.4,9.5,9.45,15.5,NaN,NaN,NaN,...,0.5,0.4,0.046848,38.930523824,-106.986967383,16.0,None,Snodgrass-2,ABLA,POINT (327770.429 4310943.085)


In [42]:
stemx = snod2_merge_gpdf.geometry.x
stemy = snod2_merge_gpdf.geometry.y
stemh = snod2_merge_gpdf['Height_Avg_M']

In [43]:
subset = snod2_merge_gpdf.head(5)

In [17]:
testfig = go.Figure(data=go.Cone(x=snod2_merge_gpdf.geometry.x,  
                             y=snod2_merge_gpdf.geometry.y, 
                             z=np.zeros(len(snod2_merge_gpdf)),
                             u=np.zeros(len(snod2_merge_gpdf)), 
                             v=np.zeros(len(snod2_merge_gpdf)), 
                             w=snod2_merge_gpdf["Height_Avg_M"],
                             colorscale='Greens',
                             anchor = 'tail',
                             sizemode = 'absolute'))

testfig.update_layout(
    title="Snodgrass-2 | Height and Location of Stems >1cm DBH",
    xaxis_title="Longitude",
    yaxis_title="Latitude",
    legend_title="Height (m)"
    )
testfig.show()

In [ ]:
snod2_19 = pd.read_csv(locdir + '/Source/Snodgrass2_inventory_2019.csv')
snod2_19 = snod2_19.rename(columns = {'Tree_Tag_#':'Tag_Number'})
snod2_19

In [ ]:
snod2_19 = snod2_19[snod2_19['Tag_Number'] != 'NT']
snod2_19['Tag_Number'] = snod2_19['Tag_Number'].astype('int64')
snod2_all = snod2_merge_gpdf.merge(snod2_19, how = 'inner', on = 'Tag_Number')

In [ ]:
snod2_all.columns
snod2_all['DBH'] = snod2_all['DBH_Avg_CM'].combine_first(snod2_all['DBH_CM'])
snod2_all

In [ ]:
plt.scatter(np.log10(snod2_all['DBH']), np.log10(snod2_all['Height_Avg_M']), marker = '+', color = 'Green')
plt.xlabel('log10 Stem DBH (cm)')
plt.ylabel('log10 Stem height (m)')
plt.title('WG-SG2 log10 DBH v. log10 Height')

In [ ]:
# library & dataset
#!pip3 install seaborn
import seaborn as sns
 
# Use the 'hue' argument to provide a factor variable
sns.lmplot('DBH', 'Height_Avg_M', data = snod2_all, fit_reg=False, hue='Sp_x', legend=False, markers = ['+', 'x', 'o' , '1'])
 
# Move the legend to an empty part of the plot
plt.legend(loc='lower right')
plt.title('Variance of DBH:Height relationship in WG-SG2 increases as DBH increases')
plt.xlabel('Stem DBH (cm)')
plt.ylabel('Stem Height (m)')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import scipy

def func(x, a, b, c):
    return a * (x**(b**-1)) + c

x = snod2_all['DBH'].iloc[0:160]
y = snod2_all['Height_Avg_M'].iloc[0:160]

yn = y + 0.2*np.random.normal(size=len(x))

popt, pcov = curve_fit(func, x, yn)

In [ ]:
plt.figure()
plt.plot(x, yn, 'ko', label="Original Noised Data")
plt.plot(x, func(x, *popt), 'r-', label="Fitted Curve")
plt.legend()
plt.show()

In [44]:
import plotly.graph_objects as go

import pandas as pd

# Read data from a csv
z_data = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/api_docs/mt_bruno_elevation.csv')

fig = go.Figure(data=[go.Surface(z=snod2_merge_gpdf['Height_Avg_M'].values)])

fig.update_layout(title='Mt Bruno Elevation', autosize=False,
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))

fig.show()